#**Analisi dei flussi degli utenti Vodafone**

Lo studio dei flussi di persone all'interno delle aree urbane è di fondamentale importanza per comprendere al meglio le criticità emergenti nella mobilità locale e per esplorare le aree di potenziale miglioramento delle infrastrutture e dei trasporti locali.<br>
La mobilità degli utenti all'interno e verso Padova è stata monitorata utilizzando i dati forniti dal gestore di telefonia mobile Vodafone, che fornisce informazioni basate sulle connessioni degli utenti alle celle della rete.<br>
I dati forniti dal carrier comprendono il monitoraggio degli utenti connessi alla rete Vodafone di Padova in un periodo di quattro mesi, da febbraio a maggio 2018.<br>
Per fornire approfondimenti statistici sul numero e sul flusso di utenti, i dati sono stati aggregati in base all'origine e ai movimenti degli utenti, calcolando la media del numero di connessioni durante il periodo di monitoraggio.<br>
Per evitare ulteriori problemi di violazione della privacy, tutte le osservazioni con meno di 30 unità (ad esempio le aree giornaliere per le quali <30 utenti hanno contribuito) sono state scartate e/o accorpate in categorie dedicate (indicate con "altro").<br>

## **Set di dati** 

I dataset si trovano al seguente link: https://www.dropbox.com/scl/fo/jra3xy77imdqc6hjc8luu/h?dl=0&rlkey=3ui6k9bhie88zxsk70zs317ow . La cartella comprende parecchi dati, si invita ad anallizzarli adeguatamente alle esigenze del problema ed eventualmente scartare dati inutili ai fini del progetto. I dati sono in diversi formati ed alcuni di questi sono descritti di seguito.

* __day_od.csv__: tabella delle origini e delle destinazioni degli utenti mediate per giorno della settimana. I dati sono forniti con il dettaglio del mese, del tipo di utente (residente a Padova/visitatore italiano/visitatore straniero), del Paese di provenienza, insieme alla provincia e al comune dell'utente (se disponibili).
* __utenti_distinti_giorno.csv__: tabella del numero di utenti distinti per provenienza. I dati sono forniti con il dettaglio del mese, del tipo di utente (residente a Padova/visitatore italiano/visitatore straniero), del Paese di provenienza, insieme alla provincia e al comune dell'utente (se disponibili).

Le informazioni sono memorizzate nei campi secondo il seguente schema: 

- MONTH: mese analizzato
- DOW: giorno analizzato
- ORIGIN: area di provenienza dell'utente
- DESTINATION: area di destinazione dell'utente
- CUST_CLASS: tipo di utente (residente / visitatore italiano / visitatore straniero)
- COD_COUNTRY: codice paese dell'utente (es. 222=Italia)
- COD_PRO: codice della provincia dell'utente (ad es. 12=Varese) 
- PRO_COM: codice del comune dell'utente (es. 12026=Busto Arsizio)
- FLOW: numero di movimenti per una determinata data-ora (con un minimo di 30 utenti)
- VISITORS: numero complessivo di utenti 

Insieme ai file di dati, vengono fornite tre tabelle di ricerca che permettono di abbinare i codici ISTAT di paese, provincia e comune ai nomi reali.

* __codici_istat_comune.csv__: file di lookup contenente la mappatura dei codici ISTAT dei comuni
* __codici_istat_provincia.csv__: file di ricerca contenente la mappatura dei codici ISTAT della provincia
* __codici_nazioni.csv__: file di lookup contenente la mappatura del codice _paese_ con il suo nome

Ulteriori informazioni, utili per lo studio dei flussi di utenti, come il numero di abitanti di ogni provincia e la distanza tra Padova e tutte le altre province italiane, possono essere estratte sulla base dei dati raccolti dall'ISTAT (es. http://dati.istat.it/Index.aspx?lang=en&SubSessionId=bcd553a8-280a-4b08-afee-cf7a72fd2ad9, http://www.istat.it/storage/cartografia/matrici_distanze/Veneto.zip).


## Assegnazione

1. Preparazione dei dati: i file csv provengono da fonti diverse e quindi presentano differenze di codifica e di fine riga che devono essere prese in considerazione nella fase di preparazione dei dati. Assicurarsi che ogni file .csv sia interpretato correttamente.
* Classifica dei visitatori provenienti da paesi stranieri: in base al numero di visitatori totali per ogni paese, creare una classifica dei primi 20 paesi con il maggior numero di visitatori.
* Classifica dei visitatori italiani per provincia, ponderata per il numero di abitanti: in base al numero di visitatori totali per provincia italiana, creare una classifica delle prime 20 province con più visitatori tenendo conto del numero di abitanti.
* Studio dei flussi di visitatori: ipotizziamo di essere chiamati a fornire indicazioni su come investire risorse per migliorare la mobilità verso Padova. Definite le tre principali direzioni dei visitatori e dei pendolari che raggiungono Padova attraverso le autostrade italiane (sud: A13 verso Bologna-Roma / ovest: A4 verso Milano-Torino / nord-est: A4 verso Venezia-Trieste), valutare quale delle tre direzioni deve essere privilegiata:
 1. Considerare un caso semplificato che coinvolge solo la mobilità di fascia media, in base al numero di visitatori/pendolari provenienti solo dalle regioni vicine
 2. Considerare le province situate sulle tre direzioni che contribuiscono maggiormente al flusso di visitatori del fine settimana e di pendolari giornalieri, effettuando uno studio più dettagliato dei flussi in base al giorno della settimana.
 3. Utilizzate i dati disponibili per fornire quella che ritenete la migliore risposta possibile. 
* Tracciare la distribuzione del numero di visitatori in base alla distanza della provincia di origine. Determinare il tipo di legge da utilizzare per descrivere la distribuzione.
 * Supponendo che si possa usare una forma analitica per descrivere l'andamento, create una regressione per stimare il numero atteso di visitatori in base alla distanza della provincia di origine. Illustrare la differenza tra la regressione ottenuta e i numeri forniti dal monitoraggio Vodafone, evidenziando le cinque discrepanze più evidenti rispetto alle aspettative.
* Utilizzate tutti i dati disponibili (ed eventualmente quelli aggiuntivi di fonte ISTAT) per estrarre un'altra informazione interessante sulla mobilità e sul flusso di visitatori e pendolari.

In [23]:
import pandas as pd

In [24]:
df_day = pd.read_csv("DATA/day_od.csv")
df_distinct_users = pd.read_csv("DATA/distinct_users_day.csv", encoding="ANSI")
df_provincie = pd.read_csv("DATA/codici_istat_provincia.csv", encoding="ANSI")
df_comuni = pd.read_csv("DATA/codici_istat_comune.csv", encoding="ANSI")
df_nazioni = pd.read_csv("DATA/codici_nazioni.csv", encoding="ANSI")

df_distance_from_padova = pd.read_csv("DATA/Veneto.txt", sep=";")

#df_day[(df_day.MONTH == "Marzo") & (df_day.DOW == "Martedì")]


In [25]:
#replace nation number with string names
nation_dictionary = {key: val for key, val in zip(df_nazioni.COD_COUNTRY, df_nazioni.COUNTRY_NAME_IT)}
provincie_dictionary = {key: val for key, val in zip(df_provincie.COD_PRO, df_provincie.PROVINCIA)}
comuni_dictionary = {key: val for key, val in zip(df_comuni.PRO_COM, df_comuni.COMUNE)}

df_day.COD_COUNTRY = df_day.COD_COUNTRY.replace(nation_dictionary)
df_day.COD_PRO = df_day.COD_PRO.replace(provincie_dictionary)
df_day.PRO_COM = df_day.PRO_COM.replace(comuni_dictionary)

df_distinct_users.COD_COUNTRY = df_distinct_users.COD_COUNTRY.replace(nation_dictionary)
df_distinct_users.COD_PRO = df_distinct_users.COD_PRO.replace(provincie_dictionary)
df_distinct_users.PRO_COM = df_distinct_users.PRO_COM.replace(comuni_dictionary)
df_distinct_users

,DOW,CUST_CLASS,COD_COUNTRY,COD_PRO,PRO_COM,VISITORS
0,Mercoledì,visitor,Italia,Reggio nell'Emilia,Reggio nell'Emilia,968
1,Lunedì,visitor,Italia,Trento,Isera,64
2,Domenica,visitor,Italia,Siena,Siena,516
3,Giovedì,visitor,Italia,Monza e della Brianza,Biassono,128
4,Giovedì,visitor,Italia,Rovigo,Villadose,512
...,...,...,...,...,...,...
12840,Mercoledì,foreigner,Moldova,NaN,NaN,176
12841,Sabato,foreigner,Egitto,NaN,NaN,164
12842,Giovedì,foreigner,Colombia,NaN,NaN,80
12843,Giovedì,foreigner,Montenegro,NaN,NaN,80


In [26]:
#primi 20 paesi stranieri con più visitatori
df_foreign_data = df_distinct_users[df_distinct_users.COD_COUNTRY != "Italia"]
grouped_df_foreign_data = df_foreign_data.groupby(by="COD_COUNTRY").sum().sort_values("VISITORS", ascending=False).iloc[:20, :]
grouped_df_foreign_data

,VISITORS
COD_COUNTRY,
Francia,129672
Svizzera,116884
Germania,109176
Cina,107796
Austria,96776
Stati Uniti d'America,76136
Spagna,71872
Paesi Bassi,70432
Romania,59012


In [27]:
"""grouped_df_foreign_data.reset_index(inplace=True)
grouped_df_foreign_data
grouped_df_foreign_data["COD_COUNTRY"] = grouped_df_foreign_data["COD_COUNTRY"].apply(lambda x: df_nazioni[df_nazioni.COD_COUNTRY == x].COUNTRY_NAME_IT.values[0])
grouped_df_foreign_data"""

'grouped_df_foreign_data.reset_index(inplace=True)\ngrouped_df_foreign_data\ngrouped_df_foreign_data["COD_COUNTRY"] = grouped_df_foreign_data["COD_COUNTRY"].apply(lambda x: df_nazioni[df_nazioni.COD_COUNTRY == x].COUNTRY_NAME_IT.values[0])\ngrouped_df_foreign_data'

In [28]:
df_distinct_users[df_distinct_users.COD_PRO == "Torino"]

,DOW,CUST_CLASS,COD_COUNTRY,COD_PRO,PRO_COM,VISITORS
43,Mercoledì,visitor,Italia,Torino,Orbassano,60
105,Martedì,visitor,Italia,Torino,Venaria Reale,64
285,Martedì,visitor,Italia,Torino,Pino Torinese,40
302,Mercoledì,visitor,Italia,Torino,Carmagnola,68
333,Lunedì,visitor,Italia,Torino,Rivalta di Torino,48
...,...,...,...,...,...,...
12657,Domenica,visitor,Italia,Torino,Chivasso,108
12673,Lunedì,visitor,Italia,Torino,Torino,3260
12757,Giovedì,visitor,Italia,Torino,Nichelino,120
12807,Mercoledì,visitor,Italia,Torino,Chivasso,52


In [29]:
"""df_population_data = pd.read_csv("DATA/DCIS_POPRES1_28032023144458463.csv")
df_population_data = df_population_data.loc[:, ["Territorio", "Value"]]
df_provincie"""

'df_population_data = pd.read_csv("DATA/DCIS_POPRES1_28032023144458463.csv")\ndf_population_data = df_population_data.loc[:, ["Territorio", "Value"]]\ndf_provincie'

In [30]:
"""df_distance_from_padova = df_distance_from_padova[df_distance_from_padova.Destinazione == "28060,00"]
df_distance_from_padova
df_comuni"""

'df_distance_from_padova = df_distance_from_padova[df_distance_from_padova.Destinazione == "28060,00"]\ndf_distance_from_padova\ndf_comuni'

In [57]:
venezia_trieste_A4 = [5, 6]
df = [df_provincie[df_provincie.COD_REG == x].PROVINCIA for x in range(5, 7)]
torino_milano_A4 = [1, 2, 3, 4, 7]
bologna_roma_A4 = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [62]:
df_day

,MONTH,DOW,ORIGIN,DESTINATION,CUST_CLASS,COD_COUNTRY,COD_PRO,PRO_COM,FLOW
0,Marzo,Domenica,108,300,visitor,Italia,Padova,Altro,493
1,Maggio,Lunedì,300,101,visitor,Italia,Pordenone,Altro,58
2,Febbraio,Sabato,108,207,visitor,Italia,Padova,Altro,39
3,Aprile,Venerdì,109,121,resident,Italia,Padova,Padova,106
4,Maggio,Giovedì,300,120,visitor,Italia,Padova,Villafranca Padovana,136
...,...,...,...,...,...,...,...,...,...
193496,Febbraio,Martedì,205,120,Altro,-999,Altro,Altro,32
193497,Marzo,Giovedì,214,115,Altro,-999,Altro,Altro,32
193498,Febbraio,Giovedì,124,121,Altro,-999,Altro,Altro,40
193499,Aprile,Domenica,110,300,visitor,Italia,Padova,Selvazzano Dentro,36


In [56]:
df_provincie[df_provincie.COD_REG == 1].PROVINCIA

0                  Torino
1                Vercelli
2                  Novara
3                   Cuneo
4                    Asti
5             Alessandria
6                  Biella
7    Verbano-Cusio-Ossola
Name: PROVINCIA, dtype: object